In [ ]:
!pip install monai

In [1]:
import os
import shutil
import glob

import pandas as pd
import numpy as np
import seaborn as sns
import cv2
import nibabel as nib
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torchmetrics.classification import BinaryJaccardIndex, Dice
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torchvision

# %% environment and functions
import matplotlib.pyplot as plt
join = os.path.join
from segment_anything import sam_model_registry
from skimage import io, transform
import monai


2024-04-15 18:15:39.594131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 18:15:39.594187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 18:15:39.595672: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# !wget -O medsam_vit_b.pth https://zenodo.org/records/10689643/files/medsam_vit_b.pth

# Dataset Class

In [4]:
class BraTSDataset(Dataset):    
    def __init__(self, data_root_folder, folder = '', n_sample=None):
        main_folder = os.path.join(data_root_folder, folder)
        self.folder_path = os.path.join(main_folder, 'slice')

    def __getitem__(self, index):
        file_name = os.listdir(self.folder_path)[index]
        sample = torch.from_numpy(np.load(os.path.join(self.folder_path, file_name)))
        img_as_tensor = np.expand_dims(sample[0,:,:], axis=0)
        mask_as_tensor = np.expand_dims(sample[1,:,:], axis=0)
        return {
            'image': img_as_tensor,
            'mask': mask_as_tensor,
            'img_id': file_name
        }
 
    def __len__(self):
        return len(os.listdir(self.folder_path))

# Load Dataset

In [3]:
data_root_folder = '/kaggle/input/full_raw - Copy'
train_dataset = BraTSDataset(data_root_folder = data_root_folder, folder = 'train')
val_dataset = BraTSDataset(data_root_folder = data_root_folder, folder = 'val')
test_dataset = BraTSDataset(data_root_folder = data_root_folder, folder = 'test')

In [5]:
BATCH_SIZE = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load MedSAM

## Self-defined functions

In [ ]:
def convert_to_binary(arr):
    return np.where(arr > 0, 1, 0)

In [7]:
def visualize_masks(nii_img):
    nii_aff  = nii_img.affine
    nii_hdr  = nii_img.header

    # print(nii_aff ,'\n',nii_hdr)
    # print(nii_data.shape)
    nii_data = nii_img.get_fdata()
    number_of_slices = nii_data.shape[1]
    number_of_frames = nii_data.shape[2]
    number_of_frames = 1

    # Define the number of columns for subplot
    num_columns = 5

    if(len(nii_data.shape)==3):
        start_slice = 80
        end_slice = 100
        num_slices = end_slice - start_slice + 1
        num_rows = num_slices // num_columns + 1

        fig, ax = plt.subplots(num_rows, num_columns, figsize=(10,10))

        for i in range(start_slice, end_slice+1):
            row = (i-start_slice) // num_columns
            col = (i-start_slice) % num_columns
            ax[row, col].imshow(nii_data[:,:,i])
            ax[row, col].axis('off')

        # Remove empty subplots
        for j in range(i-start_slice+1, num_rows*num_columns):
            fig.delaxes(ax.flatten()[j])

        plt.show()

In [8]:
def box_coordinates_from_mask(mask, margin = 5):
    # Works on [240, 240]
    # Modify for this to work on [16, 1, 240, 240]
    nonzero_indices = np.transpose(np.nonzero(mask))

    min_y = np.min(nonzero_indices[:, 0])
    max_y = np.max(nonzero_indices[:, 0])
    min_x = np.min(nonzero_indices[:, 1])
    max_x = np.max(nonzero_indices[:, 1])

    margin = 5

    x0 = min_x - margin
    y0 = min_y - margin
    x1 = max_x + margin
    y1 = max_y + margin

    # print(f"Row and column for non-zero elements with greatest x range: [{min_x}, {max_x}]")
    # print(f"Row and column for non-zero elements with greatest y range: [{min_y}, {max_y}]")
    
    box_np = np.array([[x0,y0, x1, y1]])
    box_1024 = box_np / np.array([W, H, W, H]) * 1024
    return box_1024

In [9]:
def box_coordinates_from_mask_batch(batch_mask, margin = 5):
    W = 240
    H = 240
    margin = 5
    box_1024_batch = []
    useless_image_indices = []
    
    for ind, mask in enumerate(batch_mask):
        mask = mask[0]
        nonzero_indices = torch.nonzero(mask).T
        if nonzero_indices.shape[-1] == 0:
            useless_image_indices.append(ind)
#             #box_1024 = torch.asarray([[-1, -1, -1, -1]])
        else:
            min_y = torch.min(nonzero_indices[0, :]).values
            max_y = torch.max(nonzero_indices[0, :]).values
            min_x = torch.min(nonzero_indices[1, :]).values
            max_x = torch.max(nonzero_indices[1, :]).values

            x0 = min_x - margin
            y0 = min_y - margin
            x1 = max_x + margin
            y1 = max_y + margin

            box_np = torch.asarray([[x0,y0, x1, y1]])
            box_1024 = box_np / torch.asarray([W, H, W, H]) * 1024
            box_1024_batch.append(box_1024)
    
    final_tensor = torch.cat(box_1024_batch, dim = 0)
    final_tensor = final_tensor.unsqueeze(dim = 1)
    return final_tensor, useless_image_indices

## MedSAM functions

In [10]:
# visualization functions
# source: https://github.com/facebookresearch/segment-anything/blob/main/notebooks/predictor_example.ipynb
# change color to avoid red and green
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251/255, 252/255, 30/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='blue', facecolor=(0,0,0,0), lw=2))

In [ ]:
# #%% load model and image
# MedSAM_CKPT_PATH = "/kaggle/working/medsam_vit_b.pth"
# device = "cuda:0"
# medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
# medsam_model = medsam_model.to(device)
# medsam_model.eval()

In [11]:
def box_coordinates_from_mask_batch(batch_mask, margin = 5, W = 240, H = 240):
    box_1024_batch = []
    
    for ind, mask in enumerate(batch_mask):
        mask = mask[0]
        nonzero_indices = torch.nonzero(mask).T
        
        #if(nonzero_indices.shape[-1] == 0):
        min_y = torch.min(nonzero_indices[0, :])
        max_y = torch.max(nonzero_indices[0, :])
        min_x = torch.min(nonzero_indices[1, :])
        max_x = torch.max(nonzero_indices[1, :])

        x0 = min_x - margin
        y0 = min_y - margin
        x1 = max_x + margin
        y1 = max_y + margin

        box = torch.asarray([[x0,y0, x1, y1]])
        box = box / torch.asarray([W, H, W, H]) * 1024
        box_1024_batch.append(box)
    
    box_batch = torch.cat(box_1024_batch, dim = 0)
    box_batch = box_batch.unsqueeze(dim = 1)
    return box_batch

def generate_box_and_embedding(img, mask, W = 240, H = 240):

    # resize image, make it 3D
    resizer = torchvision.transforms.Resize(size = (1024,1024))
    img = resizer(img)
    img = img.repeat(1, 3, 1, 1)
    
    #mask = resizer(mask)
    # generate bounding boxes
    box_batch = box_coordinates_from_mask_batch(mask)
  
    with torch.no_grad():
        image_embedding = medsam_model.image_encoder(img) # (1, 256, 64, 64)
        
    return image_embedding, mask, box_batch

In [12]:
#MedSAM train class
class MedSAM(nn.Module):
    def __init__(
        self,
        image_encoder,
        mask_decoder,
        prompt_encoder,
    ):
        super().__init__()
        self.image_encoder = image_encoder
        self.mask_decoder = mask_decoder
        self.prompt_encoder = prompt_encoder
        # freeze prompt encoder
        for param in self.prompt_encoder.parameters():
            param.requires_grad = False

    def forward(self, image, box):
        image_embedding = self.image_encoder(image)  # (B, 256, 64, 64)
        # do not compute gradients for prompt encoder
        with torch.no_grad():
            box_torch = torch.as_tensor(box, dtype=torch.float32, device=image.device)
            if len(box_torch.shape) == 2:
                box_torch = box_torch[:, None, :]  # (B, 1, 4)

            sparse_embeddings, dense_embeddings = self.prompt_encoder(
                points=None,
                boxes=box_torch,
                masks=None,
            )
        low_res_masks, _ = self.mask_decoder(
            image_embeddings=image_embedding,  # (B, 256, 64, 64)
            image_pe=self.prompt_encoder.get_dense_pe(),  # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings,  # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings,  # (B, 256, 64, 64)
            multimask_output=False,
        )
        ori_res_masks = F.interpolate(
            low_res_masks,
            size=(image.shape[2], image.shape[3]),
            mode="bilinear",
            align_corners=False,
        )
        return ori_res_masks

In [15]:
# Medsam Train code
MedSAM_CKPT_PATH = "/kaggle/working/medsam_vit_b.pth"
sam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
medsam_model = MedSAM(
    image_encoder=sam_model.image_encoder,
    mask_decoder=sam_model.mask_decoder,
    prompt_encoder=sam_model.prompt_encoder,
).to(device)
medsam_model.train()
medsam_model = medsam_model.to(device)
# Useless print stuff
# print(
#     "Number of total parameters: ",
#     sum(p.numel() for p in medsam_model.parameters()),
# )  # 93735472
# print(
#     "Number of trainable parameters: ",
#     sum(p.numel() for p in medsam_model.parameters() if p.requires_grad),
# )  # 93729252

# img_mask_encdec_params = list(medsam_model.image_encoder.parameters()) + list(
#     medsam_model.mask_decoder.parameters()
# )
# optimizer = torch.optim.AdamW(
#     img_mask_encdec_params, lr=1e-4, weight_decay=0.01
# )
# print(
#     "Number of image encoder and mask decoder parameters: ",
#     sum(p.numel() for p in img_mask_encdec_params if p.requires_grad),
# )  # 93729252

# Important hyperparameters and initialization
# seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction="mean")
# cross entropy loss
ce_loss = nn.BCEWithLogitsLoss(reduction="mean")
# %% train
num_epochs = 0
iter_num = 0
losses = []
best_loss = 1e10


In [ ]:
# train_dataset = NpyDataset(args.tr_npy_path)

# print("Number of training samples: ", len(train_dataset))
# train_dataloader = DataLoader(
#     train_dataset,
#     batch_size=args.batch_size,
#     shuffle=True,
#     num_workers=args.num_workers,
#     pin_memory=True,
# )

# start_epoch = 0
# if args.resume is not None:
#     if os.path.isfile(args.resume):
#         ## Map model to be loaded to specified single GPU
#         checkpoint = torch.load(args.resume, map_location=device)
#         start_epoch = checkpoint["epoch"] + 1
#         medsam_model.load_state_dict(checkpoint["model"])
#         optimizer.load_state_dict(checkpoint["optimizer"])
# if args.use_amp:
#     scaler = torch.cuda.amp.GradScaler()

# for epoch in range(start_epoch, num_epochs):
#     epoch_loss = 0
#     for step, (image, gt2D, boxes, _) in enumerate(tqdm(train_dataloader)):
#         optimizer.zero_grad()
#         boxes_np = boxes.detach().cpu().numpy()
#         image, gt2D = image.to(device), gt2D.to(device)
#         if args.use_amp:
#             ## AMP
#             with torch.autocast(device_type="cuda", dtype=torch.float16):
#                 medsam_pred = medsam_model(image, boxes_np)
#                 loss = seg_loss(medsam_pred, gt2D) + ce_loss(
#                     medsam_pred, gt2D.float()
#                 )
#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()
#             optimizer.zero_grad()
#         else:
#             medsam_pred = medsam_model(image, boxes_np)
#             loss = seg_loss(medsam_pred, gt2D) + ce_loss(medsam_pred, gt2D.float())
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()

#         epoch_loss += loss.item()
#         iter_num += 1

#     epoch_loss /= step
#     losses.append(epoch_loss)
#     if args.use_wandb:
#         wandb.log({"epoch_loss": epoch_loss})
#     print(
#         f'Time: {datetime.now().strftime("%Y%m%d-%H%M")}, Epoch: {epoch}, Loss: {epoch_loss}'
#     )
#     ## save the latest model
#     checkpoint = {
#         "model": medsam_model.state_dict(),
#         "optimizer": optimizer.state_dict(),
#         "epoch": epoch,
#     }
#     torch.save(checkpoint, join(model_save_path, "medsam_model_latest.pth"))
#     ## save the best model
#     if epoch_loss < best_loss:
#         best_loss = epoch_loss
#         checkpoint = {
#             "model": medsam_model.state_dict(),
#             "optimizer": optimizer.state_dict(),
#             "epoch": epoch,
#         }
#         torch.save(checkpoint, join(model_save_path, "medsam_model_best.pth"))

#     # %% plot loss
#     plt.plot(losses)
#     plt.title("Dice + Cross Entropy Loss")
#     plt.xlabel("Epoch")
#     plt.ylabel("Loss")
#     plt.savefig(join(model_save_path, args.task_name + "train_loss.png"))
#     plt.close()



In [ ]:
# # Medsam inference code
# @torch.no_grad()
# def medsam_inference(medsam_model, img_embed, box_1024, H, W):
#     box_torch = torch.as_tensor(box_1024, dtype=torch.float, device=img_embed.device)
#     if len(box_torch.shape) == 2:
#         box_torch = box_torch[:, None, :] # (B, 1, 4)

#     sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
#         points=None,
#         boxes=box_torch,
#         masks=None,
#     )
    
# #     print(f'img_embed: {img_embed.shape} should be (B, 256, 64, 64)')
# #     print(f'sparse_embeddings : {sparse_embeddings.shape} (B, 2, 256)')
# #     print(f'dense_prompt_embeddings : {dense_embeddings.shape} (B, 256, 64, 64)')
#     low_res_logits, _ = medsam_model.mask_decoder(
#         image_embeddings=img_embed, # (B, 256, 64, 64)
#         image_pe=medsam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
#         sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
#         dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
#         multimask_output=False,
#         )

#     low_res_pred = torch.sigmoid(low_res_logits)  # (1, 1, 256, 256)

#     low_res_pred = F.interpolate(
#         low_res_pred,
#         size=(H, W),
#         mode="bilinear",
#         align_corners=False,
#     )  # (1, 1, gt.shape)
#     low_res_pred = low_res_pred.squeeze().cpu().numpy()  # (256, 256)
#     medsam_seg = (low_res_pred > 0.5).astype(np.uint8)
#     return medsam_seg

In [23]:
valid_batch_dice = []
valid_batch_jaccard = []
dice_metric = Dice().to(device)
jaccard_index_metric = BinaryJaccardIndex().to(device)
with torch.no_grad():
    H = 240
    W = 240
    for i, batch in enumerate(tqdm(test_dataloader)):
        # get image and masks from dataloader
        imgs = batch['image'].to(device).float()
        img_masks = batch['mask'].to(device).float()
        
        indices_to_remove = []
        
        for idx, img_mask in enumerate(img_masks):
            if (torch.sum(img_mask) == 0):
                indices_to_remove.append(idx)
        
        if(len(indices_to_remove) != BATCH_SIZE):
            indices_to_select = set(range(0, BATCH_SIZE)).difference(set(indices_to_remove))
            indices_to_select = list(indices_to_select)

            imgs = imgs[indices_to_select,:,:,:]
            img_masks = img_masks[indices_to_select,:,:,:]

            image_embedding, true_mask, box = generate_box_and_embedding(imgs, img_masks)
             
            true_mask = true_mask.int()
            true_mask = true_mask[0][0]
            # medsam inference pred
#             y_pred = medsam_inference(medsam_model, image_embedding, box, H, W)

            # medsam train pred
            imgs = torch.rand(1,3,256,256).to(device).float()
            box = torch.rand(1,1,4)
            print("box shape", box.shape)
            print("imgs shape", imgs.shape)
#             imgs = imgs.repeat(1, 3, 1, 1)
            
            y_pred = medsam_model(imgs, box)
            y_pred = torch.from_numpy(y_pred)
            #y_pred = y_pred.unsqueeze(dim = 1)
            
            y_pred = y_pred.to(device)
            true_mask = true_mask.to(device)

            batch_dice_score = dice_metric(y_pred, true_mask)
            valid_batch_dice.append(batch_dice_score)

            batch_jaccard_score = jaccard_index_metric(y_pred, true_mask)
            valid_batch_jaccard.append(batch_jaccard_score)

            print(f'DICE score: {batch_dice_score}, Jaccard score: {batch_jaccard_score}', end='\r')
    print(f'Validation DICE score: {np.mean(valid_batch_dice)}, Validation Jaccard score: {np.mean(jaccard_index_metric)}', end='\r')

  0%|          | 0/28985 [00:00<?, ?it/s]

box shape torch.Size([1, 1, 4])
imgs shape torch.Size([1, 3, 256, 256])


RuntimeError: The size of tensor a (16) must match the size of tensor b (64) at non-singleton dimension 2

In [ ]:
train_loss, train_dice, train_jaccard, valid_loss, valid_dice, valid_jaccard = train_net(ddp_model, EPOCHS, train_dataloader, validation_dataloader, optimizer, loss_function)